![Callysto.ca Banner](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-top.jpg?raw=true)

<a href="https://hub.callysto.ca/jupyter/hub/user-redirect/git-pull?repo=https%3A%2F%2Fgithub.com%2Fcallysto%2Fcurriculum-notebooks&branch=master&subPath=Mathematics/StatisticalReasoning/statistical-reasoning.ipynb&depth=1" target="_parent"><img src="https://raw.githubusercontent.com/callysto/curriculum-notebooks/master/open-in-callysto-button.svg?sanitize=true" width="123" height="24" alt="Open in Callysto"/></a>

In [ ]:
from scripts.uibuttons import *
%uiButtons

import numpy as np
import pandas as pd
from scipy import stats

import plotly.offline as py
import plotly.tools as tls
from plotly import graph_objs as go

import matplotlib.pylab as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas

import ipywidgets as widgets
from IPython.display import display, Math, Latex, clear_output, HTML

# These options allow plots to display.
py.init_notebook_mode(connected=True)
fig = plt.Figure()
ax = fig.gca()
canvas = FigureCanvas(fig)

# Introduction

In this notebook we are going to explore statistics.

[Jerry Sokoloski](https://en.wikipedia.org/wiki/Jerry_Sokoloski) is a Canadian actor and former basketball player. At 7'4" (2.24 m) he is also Canada's tallest man.

![Jerry](images/jerry.jpg)

*Jerry Sokoloski walking down the street in Toronto. Photo credit: [STAN BEHAL/Toronto Sun](https://torontosun.com/author/stanbehal).*

Ok, so 7'4" seems really tall, right? What if we want to find out how Jerry *measures up* to the rest of the Canadian population of adult men? For that task, we're going to need some more information. We can find out just how out*stand*ing Jerry is by comparing his height to thousands of other men's heights in the United States, which we'll assume should be about the same as Canada.

We'll use Python to load in some data made available by the 2021 Center for Disease Control (CDC) National Health Interview Survey. Click on the code cell below, then click the `▶Run` button to download and visualize the data.

In [ ]:
import requests
from io import BytesIO
from zipfile import ZipFile
import pandas as pd
import plotly.express as px

try:
    r = requests.get('https://ftp.cdc.gov/pub/Health_Statistics/NCHS/Datasets/NHIS/2021/adult21csv.zip')
    with ZipFile(BytesIO(r.content)) as z:
        f = z.extractall()
    df = pd.read_csv('adult21.csv')
except:
    df = pd.read_csv('https://raw.githubusercontent.com/callysto/data-files/main/Mathematics/StatisticalReasoning/data/adult21.csv')

px.histogram(df['HEIGHTTC_A'], title='Adult Heights', labels={'value':'Height (inches)'}).update_layout(showlegend=False)

This **histogram** of height data, in inches, shows how many individuals in the data set were *count*ed with that height. We can see that there are some very tall individuals with heights of 96 to 99 inches (2.4 to 2.5 m). According to the [NHIS Codebook](https://ftp.cdc.gov/pub/Health_Statistics/NCHS/Dataset_Documentation/NHIS/2021/adult-codebook.pdf), those are not actual height measurments.

|Height Code|Description|
|-|-|
|96|extreme value or sex unknown|
|97|refused|
|98|not ascertained|
|99|don't know|

So we can remove those values and make a new histogram.

In [ ]:
df.drop(df[df['HEIGHTTC_A'] > 95].index, inplace=True)
px.histogram(df['HEIGHTTC_A'], title='Adult Heights', labels={'value':'Height (inches)'}).update_layout(showlegend=False)

Since we want to compare Jerry Sokoloski's height to adult male heights, let's use just the rows where `SEX_A` is `1` (male).

In [ ]:
mdf = df[df['SEX_A'] == 1]
px.histogram(mdf['HEIGHTTC_A'], title='Male Adult Heights', labels={'value':'Height (inches)'}).update_layout(showlegend=False)

For comparison, we can compare that to the female heights in the data set.

In [ ]:
px.histogram(df[df['SEX_A']==2]['HEIGHTTC_A'], title='Female Adult Heights', labels={'value':'Height (inches)'}).update_layout(showlegend=False)

## Normal Distribution

When a histogram, like these, has a large bump in the middle and smaller values ('tails') at either end, we say that the dataset is approximately **normally** distributed.

You may have heard the term [bell curve](https://en.wikipedia.org/wiki/Normal_distribution), that is *the* normal distribution. It's an idealized model for any normally distributed dataset.

## The Mean of a Distribution

The big bump in the middle of the data is the **mean** of the distribution. The mean of a normally distributed dataset is usually very close in value to the middle element. It's the average of all of the elements in the dataset. Usually, we use $\mu$ as a symbol for the mean.

### Finding the Mean Mathematically

If you want to calculate the mean for a dataset, you need to add up all of the elements and then divide by the number of elements. So, if we have the dataset ${1,2,3,4,5}$, which has 5 elements, the mean is given by

$\mu = \frac{1+2+3+4+5}{5} = \frac{15}{5} = 3$

In general, if we have the dataset ${x_1, x_2, x_3, \ldots, x_N}$, where $N$ is the number of elements, then the mean is given by

$\mu = \frac{x_1 + x_2 + x_3 + \ldots + x_N}{N} = \frac{\sum_i x_i}{N}$

Notice that we use the symbol $\sum$ as a shortcut to writing the summation of every element in the dataset.

Let's find the mean of our height data.

In [ ]:
mdf['HEIGHTTC_A'].mean()

### The Median and the Mode of a Distribution

There are two others numbers that can help us describe the distribution of a dataset. The first is the **median**, which is the **middle** number of a dataset. The **mode** is the most frequently occuring value in the dataset.

In [ ]:
mdf['HEIGHTTC_A'].median()

In [ ]:
mdf['HEIGHTTC_A'].mode()

In a normally distributed dataset, the median, mean, and mode should all be approximately equal.

## The Shape of a Distribution: Skewness

Sometimes there are elements in a dataset that can change the shape of the histogram and its underlying distribution. Let's use the heights dataset as an example. Suppose we included the heights of NBA basketball players in the dataset. This would definitely make the value of the mean higher, and might change the value of the median. Similarly, if we including the heights of children in the dataset, the mean would be lower and the median might change.

Let's plot the histogram of the heights dataset again and see what the shape looks like when children or NBA players are included.

In [ ]:
try:
    r = requests.get('https://ftp.cdc.gov/pub/Health_Statistics/NCHS/Datasets/NHIS/2018/samchildcsv.zip')
    with ZipFile(BytesIO(r.content)) as z:
        f = z.extractall()
    cdf = pd.read_csv('samchild.csv')
except:
    cdf = pd.read_csv('https://raw.githubusercontent.com/callysto/data-files/main/Mathematics/StatisticalReasoning/data/samchild.csv')
cdf.drop(cdf[cdf['CHGHT_TC'] > 95].index, inplace=True)
mcdf = cdf[cdf['SEX'] == 1]
px.histogram(pd.concat([mdf['HEIGHTTC_A'], mcdf['CHGHT_TC']]), title='Male Adult and Child Heights').update_layout(showlegend=False)

In [ ]:
try:
    # TODO: scrape NBA height data from somewhere like https://www.nba.com/players or https://github.com/simonwarchol/NBA-Height-Weight or https://craftednba.com/player-traits/length
    #ndf[['heightFeet','heightInches']] = ndf['Height'].str.split('-', expand=True).astype(int)
    #ndf['Height (inches)'] = ndf['heightFeet']*12 + ndf['heightInches']
    pass
except: 
    ndf = pd.read_csv('https://raw.githubusercontent.com/callysto/data-files/main/Mathematics/StatisticalReasoning/data/NBAheightsonly.csv',encoding='latin-1')
px.histogram(pd.concat([mdf['HEIGHTTC_A'], ndf['Height']]), title='Male Adult and NBA Player Heights').update_layout(showlegend=False)

In [ ]:
ChildHeights = pd.read_csv('https://raw.githubusercontent.com/callysto/data-files/main/Mathematics/StatisticalReasoning/data/children_heights.txt', sep='\t')
NBAheights = pd.read_csv('https://raw.githubusercontent.com/callysto/data-files/main/Mathematics/StatisticalReasoning/data/NBAheightsonly.csv',encoding='latin-1')


NBAheights = NBAheights.Height
NBAheights.columns = ['AHEIGHT']

# Reset index values for error-free joining.
NBAheights.index = range(len(heights),len(heights) + len(NBAheights))

# Combine the 'heights' dataset with the other datasets.
shortheights = heights.append(CHILDheights)
tallheights = heights.append(NBAheights)

height_histogram = go.Histogram(
    x = heights,
    histnorm = '',
    xbins=dict(
        start=50,
        end=90,
        size=1
    ),
    visible=True,
    name = 'heights'
    )

childheight_histogram = go.Histogram(
    x = shortheights,
    histnorm = '',
    xbins=dict(
        start=50,
        end=90,
        size=1
    ),
    visible=False,
    name = 'child'
)

NBAheight_histogram = go.Histogram(
    x = tallheights,
    histnorm = '',
    xbins=dict(
        start=50,
        end=90,
        size=1
    ),
    visible=False,
    name = 'NBA'
)

data1 = [height_histogram, childheight_histogram, NBAheight_histogram]

button_layer_1_height = 1.12
button_layer_2_height = 1.065

layout = go.Layout(
    title = 'Adding Data Affects the Mean and Median of a Distribution',
    width=800,
    height=650,
    autosize=False,
    xaxis = dict(
        title = 'Height',
        range = [50,90]
    ),
    yaxis = dict(title = 'Frequency')
)

height_annotations=[
    dict(text='Dataset', x=0, y=1.1, yref='paper', align='left', showarrow=False),
    dict(
        x=85,
        y=1000,
        xref='x', yref='y',
        text='Mean: {:.5}'.format(np.mean(heights)),
        showarrow = False
    ),
    dict(
        x=85,
        y=800,
        xref='x', yref='y',
       text='Median: {:.5}'.format(np.median(heights)),
       showarrow = False
    )]

child_annotations=[
    dict(text='Dataset', x=0, y=1.1, yref='paper', align='left', showarrow=False),
    dict(
        x=85,
        y=1000,
        xref='x', yref='y',
        text='Mean: {:.5}'.format(np.mean(shortheights)),
        showarrow = False
    ),
    dict(
        x=85,
        y=800,
        xref='x', yref='y',
       text='Median: {:.5}'.format(np.median(shortheights)),
       showarrow = False
    )]

NBA_annotations=[
    dict(text='Dataset', x=0, y=1.1, yref='paper', align='left', showarrow=False),
    dict(
        x=85,
        y=1000,
        xref='x', yref='y',
        text='Mean: {:.5}'.format(np.mean(tallheights)),
        showarrow = False
    ),
    dict(
        x=85,
        y=800,
        xref='x', yref='y',
       text='Median: {:.5}'.format(np.median(tallheights)),
       showarrow = False
    )]

updatemenus=list([
    dict(
        active=0,
        buttons=list([
            dict(
                args=[{'visible': [True, False, False]},{'annotations': height_annotations}],
                label='Heights',
                method='update'
            ),
            dict(
                args=[{'visible': [False, True, False]},{'annotations': child_annotations}],
                label="Heights with children's heights",
                method='update'
            ),
            dict(
                args=[{'visible': [False, False, True]},{'annotations': NBA_annotations}],
                label='Heights with NBA heights',
                method='update'
            )                     
        ]),
        direction = 'down',
        pad = {'r': 10, 't': 10},
        showactive = True,
        x = 0.1,
        xanchor = 'left',
        y = button_layer_1_height,
        yanchor = 'top'            
    )
])

annotations = list([
    dict(text='Dataset', x=0, y=1.1, yref='paper', align='left', showarrow=False )
])
layout['updatemenus'] = updatemenus

fig = dict(data=data1, layout=layout)
py.iplot(fig)

<ul> 
    <li> What happens to the median as the children's or NBA heights are included in the histogram? </li>
    <li> When a statistic doesn't change based on different input we say it is **stable**. Is the median stable? Is the mean? </li>
</ul>

Adding in the children's heights made the big bump in the histogram shift to the right. The children's heights also caused the mean to decrease but the median to stay the same. When a distribution has its bump shifted to the right like this, we say that the distribution is **skewed left**, **left-skewed**, or **negatively skewed**.

When we added the NBA heights, the big bump in the middle shifted to the left. The NBA heights caused the mean to increase but the median to stay the same.  When a distribution has its bump shifted to the left like this, we say that the distribution is **skewed right**, **right-skewed**, or **positively skewed**.

Leaving out both the children's heights and the NBA heights, the distribution looks roughly **symmetric**. This means that the distribution is not skewed left or right, but looks roughly as you look away from the middle bump in either direction.

In general, we have the following:
<ul>
    <li> If the median is higher than the mean, the distribution is <b>skewed right</b>. </li>
    <li> If the mean is higher than the median, the distribution is <b>skewed left</b>. </li>
    <li> If the median and the mean are approximately equal, the distribution is <b>symmetric</b>. </li>
</ul>

### Outliers

Sometimes, if there are only a few data elements at either end of the histogram, the data elements causing the skewness might be **outliers**. An **outlier** is a data element that could be present for a number of reasons, such as a mistake when recording the data, misreading the data, or an extremely rare observation.

## The Shape of a Distribution: Standard Deviation

The mean, median, and mode are good indicators of how normal a distribution or dataset might be, but they don't tell us the whole story. If we want to know about how spread out the elements of a dataset are, then we need a new statistic. The standard deviation of a dataset, written as $\sigma$, is the measurement of how much the elements of the dataset vary as a group. In other words, how spread out the histogram is.

### *Exercise*

Move the slider below to see how different standard deviations affect the shape of the normal distribution.

In [ ]:
N = 250

# Array of standard deviations and random dataset.
sigmas = np.linspace(0.1,1,N)
randoms = np.random.randn(N)

data = [go.Histogram(x = np.sqrt(s)*randoms,
        histnorm = '',
        xbins=dict(
            start=-4.0,
            end=4.0,
            size=0.08
        ),
        visible=False
      ) for s in sigmas]

steps = []
for i in range(N):
    step = dict(
        method='restyle',
        args=['visible', [False] * N],
        label='sigma = {:.3}'.format(sigmas[i])
    )
    step['args'][1][i] = True
    steps.append(step)

sliders = [dict(
    active=1,
    steps=steps
)]

layout = dict(
        title = 'Effect of Changing the Standard Deviation on the Shape of a Distribution',
        sliders=sliders, 
        xaxis = dict(
            range = [-4,4],
            title = 'Standardized Values'
        ),
        yaxis = dict(
            range = [0,30],
            title = 'Frequency'
        )
    )

data[0].visible = True

fig = dict(data=data, layout=layout)
py.iplot(fig)

<ul> 
    <li> What happens to the spread of the histogram when $\sigma$ is small? </li>
    <li> What happens to the spread of the histogram when $\sigma$ is large? </li>
    <li> What can you estimate will happen for very small or very large values of $\sigma$? </li>
</ul>

### Back to Jerry Sokoloski

Using the standard deviation, we can also find out how far a single data element is from the rest of the dataset. We're going to use the standard deviation ($\sigma$) of the heights dataset to find out exactly how different Mr. Sokoloksi's height is from the population. First, we'll use a line of code to find the value of $\sigma$ for the heights dataset.

In [ ]:
sigma = np.std(heights)

print('The standard deviation of the Heights dataset is {:.4} inches.'.format(sigma))

One more thing about the standard normal distribution: we can expect that a certain proportion of elements will be within a given distance from the mean. These distances are given by $\sigma$. Look at the graphic below to see the proportion of elements within $\sigma$, $2\sigma$, and $3\sigma$ of the mean.

![normal](images/normaldist.png)
*LaTeX code adapted from John Canning: http://johncanning.net/wp/?p=1202.*

This graphic shows that 68% of the values in a normally distributed dataset will be at most a distance of $\sigma$, or one standard deviation, from the mean. It also shows that 95% of the data should be within 2$\sigma$ (2 standard deviations) of the mean, and that 99.7% of the data should be within 3$\sigma$ (3 standard deviations) of the mean. This is often called the **68-95-99.7 rule**. 

The 68-95-99.7 rule can also be seen as the area under the curve that defines the normal distribution. 100% of the elements in a normally distributed dataset will be under the curve. We then have 

<ul>
    <li> 68% of the data elements represent the area under the normal curve between $-\sigma$ and $\sigma$, </li>
    <li> 95% of the data elements represent the area under the normal curve between $-2\sigma$ and $2\sigma$, and </li>
    <li> 99.7% of the data elements represent the area under the normal curve between $-3\sigma$ and $3\sigma$. </li>
</ul>

What this means in context of our example is that we expect 68% of all men's heights to be within $\sigma = 2.787$ of the mean. Remember, the mean of our dataset was $\mu = 69.872$. 

Therefore, 
<ul>
    <li>68% of Canadian men should have a height between $69.872 - 2.787 = 67.085$ inches and $69.872 + 2.787 = 72.659$ inches, </li>
    <li>95% of Canadian men should have a height between $69.872 - 2\times2.787 = 64.298$ inches and $69.872 + 2\times2.787 = 75.446$ inches, and</li>
    <li>99.7% of Canadian men should have a height between $69.872 - 3\times2.787 = 61.511$ inches and $69.872 + 3\times2.787 = 78.233$ inches. </li>
</ul>

For example, a height of 79 inches (6'7" or 2.01 m) is over 3 standard deviations higher than the mean. Since 99.7% of the population should be *within* 3 standard deviations of the mean, we can subtract 100-99.7 to see that 0.3% of male heights should be 3$\sigma$ below the mean **and** 3$\sigma$ above the mean.

Since 79 inches is more than 3$\sigma$ above the mean, we can say that his height is somewhere in the top 0.15 % of male heights in Canada. 

We got the number 0.15 %; by dividing 0.3 %; in half, since we only were looking at the heights in the region 3$\sigma$ above the mean.

## Comparing Apples and Oranges: Z-Scores

Canadian Tire is a well-known retailer with stores across Canada. In November of 2014, Canadian Tire stock prices jumped from 198.80 dollars to 240.00 dollars, a substantial increase of &dollar;41.20, or almost 21&percnt;! For any investor, a 21&percnt; increase would be extremely welcome and rare.

So what's more exceptional, Jerry Sokoloski's height or Canadian Tire's stock jump in 2014? We can check this by **standardizing** the datasets using a **z-score**.

The **z-score** of a data element is found by subtracting the mean from the element and dividing by the standard deviation. In other words, the z-score of the element $x$ is given by
$$
z = \frac{x - \mu}{\sigma}.
$$

The **z-score** is a way of locating a standardized data element on the idealized normal distribution for a normally distributed dataset.

We will use Python to calculate the z-scores of the heights dataset.

In [ ]:
z_heights = (heights - mean)/sigma

print('Standardized Heights dataset with: \nmean = {:.5} inches \nstandard deviation = {:.4} inches'.format(mean,sigma))
print(z_heights)

Standardizing a normally distributed dataset converts the data elements to their corresponding locations on the idealized bell curve, and gives the standardized dataset a mean of 0 and standard deviation of 1.

Now we'll load in the Canadian Tire stock dataset, standardize it, and plot the histogram.

In [ ]:
# Read in the Canadian Tire financial data.
cantire = pd.read_csv('https://raw.githubusercontent.com/callysto/data-files/main/Mathematics/StatisticalReasoning/data/canadiantire.txt', sep = '\t')

# Gains are the closing stock value minus the opening stock value.
gain = cantire.Close - cantire.Open

# We'll remove the highest stock gain for this example.
high_gain = gain.max()
gain = gain[gain < high_gain]

# Standardize the dataset.
z_gain = (gain - np.mean(gain))/np.std(gain)

print('Canadian Tire dataset loaded with {} entries.'.format(len(z_gain)))
print('Canadian Tire gain data standardized.')

# Plot the histogram.
hist_data1 = [go.Histogram(
                x = z_gain, 
                visible = True
            )]

layout1 = dict(
    xaxis = dict(
        range = [np.min(z_gain)-1,np.max(z_gain)+1],
        title = 'Standardized Stock Gains'
            ),
    yaxis = dict(
        title = 'Frequency'
        ),
    title='Histogram of Standardized Canadian Tire Stock Gains'
        )

fig = dict(data=hist_data1, layout=layout1)
py.iplot(fig)

It looks like the Canadian Tire stock dataset is roughly normally distributed.

Now we'll compare Mr. Sokoloski's height to the greatest Canadian Tire stock value jump. Here is where z-scores really shine: *the units don't matter!* Remember, our heights dataset had inches as the unit of measurement. The stock dataset uses stock prices as units. When we standardize both datasets, the units are cancelled and we are left with *dimensionless* numbers.

Another benefit to using z-scores is that they enable us to determine the distance a particular data element is from the mean. We can sometimes use the 68-95-99.7 rule to find out the proportion of the dataset with values above or below the value of the data element.

Let's find the z-scores for Jerry Sokoloski's height and the biggest stock value increase.

In [ ]:
# Standardizing Jerry's height.
z_jerry = (88 - np.mean(heights))/np.std(heights)

# Standardizing the height of the tallest spruce.
z_gain = (high_gain - np.mean(gain))/np.std(gain)

print("The z-score for Jerry's height is {:.5}.".format(z_jerry))
print("The z-score for the Canadian Tire stock gain is {:.5}.".format(z_gain))

These z-scores tell us how many standard deviations each standardized value is from the mean of the idealized normal distribution. Jerry's height is 6.5049 standard deviations above the mean, and the Canadian Tire stock jump is 5.2857 standard deviations above the mean.

This tells us that Jerry's height is more exceptional and rare than the Canadian Tire stock jump, though both are very exceptional. It's difficult to use the 68-95-99.7 rule here, because neither z-score is exactly 1, 2, or 3 standard deviations from the mean. However, Python has a function for finding the proportion of the population below or above z-scores that lie in between standard deviations.

We call the probability of the location of a z-score its **p-value**.

In [ ]:
# Calculate p-values for Jerry and Canadian Tire z-scores.
p_jerry = 1 - stats.norm.cdf(z_jerry, 0, 1)   # Subtract the proportion below z_jerry from 1 = 100%
p_gain = 1 - stats.norm.cdf(z_gain, 0, 1)     # Subtract the proportion below z_gain from 1 = 100%

print("The probability of observing Jerry's height in the population is {:.6} %.".format(p_jerry*100))
print("The probability of observing the Canadian Tire stock jump is {:.6} %.".format(p_gain*100))

Again, we see that it is far less likely to observe Jerry's height (a 0.0000000039% probability) than to observe the Canadian Tire stock jump (a 0.0000063% probability).

# Summary

When given a dataset, we can check if it is normally distributed by:
<ul>
    <li> visually checking its plotted histogram or </li>
    <li> calculating and comparing the mean, the median, and the mode. </li>
</ul>

The median and the mean can tell us if the distribution is skewed left or right, which may indicate outliers in the dataset.

Examining the shape of the histogram can tell us about the skewness of the underlying distribution, and may indicate that we have outliers in the dataset. The standard deviation tells us about the spread of the data.

We can compare two different normally distributed datasets by standardizing both datasets. This transforms the distribution for both datasets into the idealized normal distribution. Once we have the datasets in terms of the normal distribution, we can find the probability of observing a given data element by either the 68-95-99.7 rule or using technology such as Python.

# *Exercises*
<ol>
    <li> Facebook stock prices change over time, depending on many factors. Below is a plot of the histogram of Facebook stock gains between the years of 2013-2018. Look at the histogram to check if this dataset is normally distributed. </li>
</ol>

In [ ]:
# load in the data.
facebook = pd.read_csv('https://raw.githubusercontent.com/callysto/data-files/main/Mathematics/StatisticalReasoning/data/facebook.csv')

# Select the difference between opening price and closing price: the 'Gain'.
facebook = facebook.Gain

# Plot the histogram.
fb_data = [go.Histogram(
            x = facebook,
            visible = True
)]

layout = dict(
    title = 'Histogram of Facebook Stock Gains',
    xaxis = dict(
        range = [np.min(facebook)-1,np.max(facebook)+1],
        title = 'Stock Gains'
            ),
    yaxis = dict(
        title = 'Frequency'
            )
        )

fig = dict(data = fb_data, layout = layout)
py.iplot(fig)

In [ ]:
from ipywidgets import interact_manual, widgets, Layout
      
@interact_manual(answer = widgets.Select(
    options = ['not normally distributed.', 'normally distributed.'],
    value = None,
    rows = 2,
    description = 'The data is: ',
    disabled=False
))
def get_answer_one(answer):
    if answer == 'normally distributed.':
        print('Correct!')
    else:
        print('Look again at the shape of the histogram. Does it look at all like a bell?') 
        
# Change default 'Run Interact' button text.
get_answer_one.widget.children[1].description = 'Check Answer'

<ol start='2'>
    <li> Look at the calculated mean, median, and mode for the Facebook dataset. What do these numbers tell you about the distribution of the data? </li>
</ol>

In [ ]:
fb_mean = np.mean(facebook)
fb_median = np.median(facebook)
fb_mode = stats.mode(facebook, axis = None)

print('Mean: ${} \nMedian: ${}\nMode: ${}'.format(fb_mean, fb_median, fb_mode[0][0]))

In [ ]:
@interact_manual(answer = widgets.Select(
    options = ['normally distributed.', 'not normally distributed.'],
    value = None,
    rows = 2,
    description = 'The data is: ',
    disabled=False
))
def get_answer_two(answer):
    if answer == 'normally distributed.':
        print('Correct!')
    else:
        print('Look again at the mean and median. Are they close in value or are they very different?') 

# Change default 'Run Interact' button text.
get_answer_two.widget.children[1].description = 'Check Answer'

<ol start = '3'>
    <li> The standard deviation for the Facebook dataset is displayed below. The largest loss in Facebook stock prices during the years 2013-2018 was a drop of \$9.43 on 8 February 2018. Calculate the z-score for this loss by using the formula $z=\frac{x - \mu}{\sigma}$. </li>
</ol>

In [ ]:
fb_stdev = np.std(facebook)
print('The standard deviation for the Facebook data is {}.'.format(fb_stdev))

# (9.43 - fb_mean)/fb_stdev

In [ ]:
@interact_manual(answer = widgets.BoundedFloatText(
    value=0,
    min=0,
    max=10.0,
    step=0.0000000000000001,
    description='z-score:',
    disabled=False
))
def get_answer_three(answer):
    if 6.3 <= answer <= (9.43-fb_mean)/np.std(facebook):
        print('Correct!')
    else:
        print('Check your algebra carefully, and try rounding to 3 digits.') 
        
# Change default 'Run Interact' button text.
get_answer_three.widget.children[1].description = 'Check Answer'

<ol start = '4'>
    <li> By examining the z-score that you just calculated for the largest stock drop, determine if this loss is more exceptional than Jerry Sokoloski's height by using the p-value calculator below. <br><b>Hint:</b> Jerry's z-score was 6.5049. </li>
</ol>

In [ ]:
style = {'description_width': 'initial'}

@interact_manual(answer = widgets.FloatText(
    value=0,
    description='Facebook z-score:',
    style = style,
    disabled=False
))
def calculate_answer_four(answer):
    p = 1 - stats.norm.cdf(answer, 0, 1)
    print('The probability of observing the Facebook stock drop of {} is {}%'.format(answer, p*100))
    
# Change default 'Run Interact' button text.
calculate_answer_four.widget.children[1].description = 'Calculate p-value'

@interact_manual(answer = widgets.Select(
    options = ["less exceptional than Jerry's height.", "more exceptional than Jerry's height."],
    description = 'Based on the above p-value, the Facebook stock drop is ',
    value = None,
    disabled=False,
    style=style,
    layout=Layout(width='75%', height='80px')
))
def answer_fourb(answer):
    if answer == "less exceptional than Jerry's height.":
        print("Yes!")
    else:
        print("Try again, checking that you got the right z-score in question 3.")

# Change default 'Run Interact' button text.
answer_fourb.widget.children[1].description = 'Check Answer'

# Conclusion



[![Callysto.ca License](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-bottom.jpg?raw=true)](https://github.com/callysto/curriculum-notebooks/blob/master/LICENSE.md)